This code loads files, and gets them ready for experiment. The data used in this first part is the ground based measurements. 

In [ ]:
import re
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# === LIST YOUR FILES HERE (up to 3, or fewer) ===
file_paths = [
    './ionosphere_central/vTEC_data/uqrg1000.24i',
    './ionosphere_central/vTEC_data/uqrg1010.24i'
]
# Empty strings or missing files will be skipped

def parse_ionex_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    header_end = [i for i, l in enumerate(lines) if 'END OF HEADER' in l][0]
    header = lines[:header_end]

    for line in header:
        if 'LAT1 / LAT2 / DLAT' in line:
            lat1, lat2, dlat = map(float, line.split()[:3])
        if 'LON1 / LON2 / DLON' in line:
            lon1, lon2, dlon = map(float, line.split()[:3])

    lats = np.arange(lat1, lat2 - 0.1, -abs(dlat))  # top-to-bottom
    lons = np.arange(lon1, lon2 + 0.1, dlon)

    maps = []
    epochs = []
    i = header_end + 1
    while i < len(lines):
        if 'START OF TEC MAP' in lines[i]:
            epoch_line = lines[i+1]
            y, mo, d, h, mi, s = map(int, epoch_line[:36].split())
            if h == 24:
                h = 0
                base_date = datetime(y, mo, d) + timedelta(days=1)
                epoch = datetime(base_date.year, base_date.month, base_date.day, h, mi, s)
            else:
                epoch = datetime(y, mo, d, h, mi, s)
            epochs.append(epoch)

            grid = []
            i += 2
            while i < len(lines) and ('START OF TEC MAP' not in lines[i]) and ('END OF FILE' not in lines[i]):
                if 'LAT/LON1/LON2/DLON/H' in lines[i]:
                    row = []
                    i += 1
                    while i < len(lines):
                        line_strip = lines[i].strip()
                        if not line_strip or not re.match(r'^[-\d\s]+$', line_strip) or 'END OF TEC MAP' in line_strip:
                            break
                        vals = [float(v) if v != '9999' else np.nan for v in line_strip.split()]
                        row.extend(vals)
                        i += 1
                    if len(row) != len(lons):
                        print(f"Warning: Row length {len(row)} != number of longitudes {len(lons)} at epoch {epoch}. Padding/truncating.")
                        row = (row + [np.nan]*len(lons))[:len(lons)]
                    grid.append(row)
                else:
                    i += 1
            if len(grid) != len(lats):
                print(f"Warning: Grid row count {len(grid)} != number of latitudes {len(lats)} at epoch {epoch}. Padding/truncating.")
                while len(grid) < len(lats):
                    grid.append([np.nan]*len(lons))
                grid = grid[:len(lats)]
            grid = np.array(grid)
            maps.append(grid)
        else:
            i += 1

    maps = np.array(maps)  # shape: (n_epochs, n_lat, n_lon)
    data = {}
    for lat_idx, lat in enumerate(lats):
        for lon_idx, lon in enumerate(lons):
            data[(lat, lon)] = maps[:, lat_idx, lon_idx]
    df = pd.DataFrame(data, index=epochs)
    df.columns = pd.MultiIndex.from_tuples(df.columns, names=['lat', 'lon'])
    return df

# Combine all files into a single DataFrame
dfs = []
for fp in file_paths:
    if fp:  # Only try non-empty file paths
        try:
            print(f"Processing {fp} ...")
            dfs.append(parse_ionex_file(fp))
        except Exception as e:
            print(f"Failed to process {fp}: {e}")

if not dfs:
    raise ValueError("No valid files found!")

# Concatenate and deduplicate/sort
combined_df = pd.concat(dfs)
combined_df = combined_df[~combined_df.index.duplicated(keep='first')]
combined_df = combined_df.sort_index()

print(combined_df.head())
print(f"\nCombined shape: {combined_df.shape}, columns: {len(combined_df.columns)} (lat-lon grid points), times: {len(combined_df.index)}")

# Save for future use
combined_df.to_parquet("combined_ionex_vtec.parquet")
